# Test Docker environments

After having created the two Docker environments, you will perform a simple test to attest these Docker images can be spawned on an AML compute node. You will execute a simple python script that returns basic diagnostic values.

In production, you should considerr an automated build/test pipeline.

## Goal

The goal of this notebook is two-fold:
1. Test the successful creation of your Docker environments
2. Demonstrate how to run a python script in your newly created Docker environment



### 1 | Create a compute target and check install
These cells are a pre-flight check to ensure that you've got the necessary requirements accessible and that a compute cluster exists. Like the subsequent notebooks, this is intende to be run on an AzureML compute instance.

You will create a new cluster named `testcluster` composed of up to 2 nodes of `Standard_NC4as_T4_v3`. These settings will be stored in a dictionary called `config`.

In [ ]:
import azureml.core
workspace = azureml.core.Workspace.from_config()

config = {}
config["compute_size"] = "STANDARD_NC4AS_T4_v3"
config["compute_target"] = "testcluster"
config["compute_node_count"] = 2
config["pytorch_configuration"] = {
    "node_count": 2, # num of computers in cluster
    "process_count": 2} # gpus-per-computer * node_count
config["training_command"] = "python diagnose_environment.py"
config["experiment"] = "Testing_Axolotl_images"
config["source_directory"] = "src"

In [ ]:
try:
    cluster = azureml.core.compute.ComputeTarget(
        workspace=workspace, 
        name=config['compute_target']
    )
    print('Found existing compute cluster')
except azureml.core.compute_target.ComputeTargetException:
    compute_config = azureml.core.compute.AmlCompute.provisioning_configuration(
        vm_size=config['compute_size'],
        max_nodes=config['compute_node_count']
    )
    cluster = azureml.core.compute.ComputeTarget.create(
        workspace=workspace,
        name=config['compute_target'], 
        provisioning_configuration=compute_config
    )
    
cluster.wait_for_completion(show_output=True)

### 2 | Running the Diagnostics on both environments, in sequence

In this step, you will retrieve the environments you have created in the previous notebook. You will then submit 2 jobs - one after the other - running the same command in both. The only difference is the environment.
You can observe the output of these commands in the AzureML UI for ease of evluation. You could also retrieve the output from the SDK or the CLI.

Connect to (or create) the experiment that will host the training run we'll launch. A single experiment can host many runs, each exploring a different set of parameters, architecture, or other approach to a the same problem. Metrics from multiple runs within a single experiment can be plotted against each other in AzureML studio.

In [ ]:
experiment = azureml.core.Experiment(workspace, config['experiment'])

For each environment, create a `ScriptRunConfig` object that determines what should be executed (in our case, `src/diagnost_environment.py`) and submit it as an Experiment.

In [ ]:
environments = ["axolotl_gold", "axolotl_acpt"]
for environment in environments:
    distributed_job_config = azureml.core.runconfig.PyTorchConfiguration(**config['pytorch_configuration'])
    aml_config = azureml.core.ScriptRunConfig(
                source_directory=config['source_directory'],
                command=config['training_command'],
                environment=azureml.core.Environment.get(workspace, name=environment),
                compute_target=config['compute_target'],
                distributed_job_config=distributed_job_config,
        )
    run = experiment.submit(aml_config)
    run.set_tags({
        "environment":environment
    })

    print(f"View run details:\n{run.get_portal_url()}")

This concludes this notebook. By executing the steps above, you have submitted two jobs to a cluster composed of 2 nodes. The difference between both jobs is that one of them uses a gold image `axolotl_gold`, while the other uses an image based on ACPT `axolotl_acpt`.

The 2nd image ships with Nebula, while the first does not.

When you submitted each job, it was executed across all nodes in the cluster. This means that you will find 2 std_out in the logs of the job. The image below shows the output of running the gold image diagnostic.

![Logs of one of the jobs](img/axolotl_gold.png)

This one demonstrates the outcome of the 2nd image diagnostics. The Axolotl_ACPT job differs by having Nebula loaded:
![Logs of one of the jobs](img/axolotl_acpt.png)